In [1]:
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import anndata

In [ ]:
%%R 

library("igraph")

In [2]:
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

c:\Users\saman\anaconda3\envs\gobi_env\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [ ]:
%%R
#install.packages("SoupX")
library(SoupX)

In [32]:
# read in CellRanger output data

folders = ["CID005217-1", "CID005221-1", "CID006493-1", "CID006515-1", "CID007664-1", "CID007827-1", "CID008474-1"]

# read sample
sample = folders[0]
path = 'C:/Users/saman/00_Bachelorarbeit/BA_code/data/' + sample + '/filtered/'
adata = sc.read_mtx(
        filename= (path + 'matrix.mtx'),  # the .mtx file 
)
adata = adata.transpose()

# obs = single cells and their barcodes (2312)
# var = genes and their annotations (33808)

adata.obs['sampleID'] = sample
adata.obs['barcode'] = pd.read_csv((path + 'barcodes.tsv'), delimiter='\t', names=['barcode']).values
adata.var = pd.read_csv((path + 'features.tsv'), delimiter='\t', names=['gene_ids', 'index', 'feature_types'], index_col='index')
len(adata.obs)

2312

In [36]:
adata.obs

,sampleID,barcode
0,CID005217-1,AAACCCAGTATTTCGG
1,CID005217-1,AAACGAACAATTGGTC
2,CID005217-1,AAACGAACATTCAGGT
3,CID005217-1,AAACGCTAGCAGTAAT
4,CID005217-1,AAACGCTTCTAGGCAT
...,...,...
2307,CID005217-1,TTTGGAGAGCCGTTAT
2308,CID005217-1,TTTGGAGGTAGCACAG
2309,CID005217-1,TTTGGAGTCATGAAAG
2310,CID005217-1,TTTGGTTAGTGCAGGT


In [37]:
# some QC 

# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)

In [38]:
adata.var

,gene_ids,feature_types,mt,ribo,hb,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts
index,,,,,,,,,,,
MIR1302-2HG,ENSG00000243485,Gene Expression,False,False,False,0,0.000000,0.000000,100.000000,0.0,0.000000
FAM138A,ENSG00000237613,Gene Expression,False,False,False,0,0.000000,0.000000,100.000000,0.0,0.000000
OR4F5,ENSG00000186092,Gene Expression,False,False,False,0,0.000000,0.000000,100.000000,0.0,0.000000
AL627309.1,ENSG00000238009,Gene Expression,False,False,False,14,0.006055,0.006037,99.394464,14.0,2.708050
AL627309.3,ENSG00000239945,Gene Expression,False,False,False,1,0.000433,0.000432,99.956747,1.0,0.693147
...,...,...,...,...,...,...,...,...,...,...,...
AC233755.2,ENSG00000277856,Gene Expression,False,False,False,1,0.000865,0.000865,99.956747,2.0,1.098612
AC233755.1,ENSG00000275063,Gene Expression,False,False,False,1,0.000433,0.000432,99.956747,1.0,0.693147
AC240274.1,ENSG00000271254,Gene Expression,False,False,False,80,0.035900,0.035270,96.539792,83.0,4.430817


## SoupX 